In [ ]:
import time
import re
import pandas as pd
import csv

import requests
from bs4 import BeautifulSoup as bs
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.660 YaBrowser/23.9.0.0 Safari/537.36',
    'Host': 'chelyabinsk.hh.ru',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Language': 'ru,en-US;q=0.7,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate, br',
    'DNT': '1',
    'Connection': 'keep-alive'
}
sess = requests.Session()
sess.headers.update(headers)

In [ ]:
driver = uc.Chrome(headless=True,use_subprocess=False)

## Get URLs

In [ ]:
def get_urls(start_html, num_of_vacancy):

    start_urls = list(set(re.findall('http[s]?://chelyabinsk.hh.ru/vacancy/[0-9]+', start_html)))
    
    while len(start_urls) < num_of_vacancy:
    
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        button_next = driver.find_element(By.LINK_TEXT, 'дальше')
        button_next.click()
        time.sleep(3)
        html = driver.page_source
        current_urls = re.findall('http[s]?://chelyabinsk.hh.ru/vacancy/[0-9]+', html)
        for url in current_urls:
            start_urls.append(url)
        print('Get unique URLS: ', len(set(current_urls)))
        
    all_urls = list(set(start_urls))

    return all_urls

In [ ]:
location_codes = []

In [ ]:
# получаю коды локаций в которых больше 2к резюме

for i in range(1,113):
    url = f"https://chelyabinsk.hh.ru/search/vacancy?L_save_area=true&items_on_page=100&area={i}&search_field=name&search_field=company_name&search_field=description&text=&enable_snippets=false"
    response = sess.get(url)
    page = bs(response.text, 'html.parser')
    find_result = page.find('h1', class_="bloko-header-section-3")
    try:
        num_of_vacancy = find_result.text[8:-9]
        num_of_vacancy = re.sub(r'\D', '', num_of_vacancy)
        if num_of_vacancy:
            if int(num_of_vacancy) > 2000:
                location_codes.append(i)
                print(num_of_vacancy)
    except:
        pass

In [ ]:
all_urls = []

for i in location_codes:
    
    try:
        url = f"https://chelyabinsk.hh.ru/search/vacancy?L_save_area=true&items_on_page=100&area={i}&search_field=name&search_field=company_name&search_field=description&text=&enable_snippets=false"
        driver.get(url)
        html = driver.page_source
        urls = get_urls(html, 3900)
        all_urls.append(urls)
    except:
        pass
        

In [ ]:
all_urls_1d = []

for i in all_urls:
    for j in i:
        all_urls_1d.append(j)

In [ ]:
all_urls = list(set(all_urls_1d))

In [ ]:
len(all_urls)

In [ ]:
urls_series = pd.Series(all_urls)
urls_series.to_csv('data/hh_vacancies_urls.csv', index=False)

## Get data

In [ ]:
with open('data/hh_vacancies_urls.csv', newline='') as f:
    reader = csv.reader(f)
    all_urls = list(reader)

In [ ]:
len(all_urls)

In [ ]:
df = pd.DataFrame({
    'id':[],
    'title':[],
    'description':[],
    'salary':[],
    'currency':[],
    'compensation_type':[]
})

In [ ]:
for n, url in enumerate(all_urls_1d):
    
    response = sess.get(url)
    page = bs(response.text, 'html.parser')
    id = url[-8:]
    dirty_description = page.find('div', class_="g-user-content")
    description = re.sub(r'(\<(/?[^>]+)>)', '', str(dirty_description))
    dirty_title = page.find('h1', class_="bloko-header-section-1")
    title = re.sub(r'(\<(/?[^>]+)>)', '', str(dirty_title))
    dirty_salary = page.find('span', class_="bloko-header-section-2 bloko-header-section-2_lite")
    salary = re.sub(r'(\<(/?[^>]+)>)', '', str(dirty_salary))
    df.loc[n, 'id'] = id
    df.loc[n, 'title'] = title
    df.loc[n, 'description'] = description
    df.loc[n, 'salary'] = salary
    
    if n % 2000 == 0:
        df.to_csv(f'data\my_data_hh_{n}.csv', index=False)
    
    print(n)

In [ ]:
df